In [1]:
!pip install pygame
import pygame
import time

# tracking stuff
!pip install  opencv-python mediapipe sklearn matplotlib 
#tensorflow-gpu==2.4.1
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp

# sound stuff
!pip install pyaudio
!pip install pysinewave
import time
from pysinewave import SineWave
import pyaudio as p
import math
import threading

mp_holistic = mp.solutions.holistic # Holistic model
mp_drawing = mp.solutions.drawing_utils # Drawing utilities

def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # COLOR CONVERSION BGR 2 RGB
    image.flags.writeable = False                  # Image is no longer writeable
    results = model.process(image)                 # Make prediction
    image.flags.writeable = True                   # Image is now writeable 
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # COLOR COVERSION RGB 2 BGR
    return image, results

def draw_landmarks(image, results):
    # mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION) # Draw face connections
    # mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS) # Draw pose connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw right hand connections

def draw_styled_landmarks(image, results):
    # # Draw face connections
    # mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION, 
    #                          mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1), 
    #                          mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
    #                          ) 
    # # Draw pose connections
    # mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
    #                          mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4), 
    #                          mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
    #                          ) 
    # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                             ) 
    # Draw right hand connections  
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                             ) 





     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.9/12.9 MB 7.0 MB/s eta 0:00:0000:0100:01
pygame 2.3.0 (SDL 2.24.2, Python 3.9.16)
Hello from the pygame community. https://www.pygame.org/contribute.html
Could not import the PyAudio C module 'pyaudio._portaudio'.


ImportError: dlopen(/Users/alexsilcock/Documents/GitHub/bathhack23/.conda/lib/python3.9/site-packages/pyaudio/_portaudio.cpython-39-darwin.so, 0x0002): symbol not found in flat namespace '_PaMacCore_SetupChannelMap'

load models

In [ ]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard

def extract_keypoints(results):
    # face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3)
    # pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return lh, rh


actions = np.array(['hand', 'fist', 'thumbs_up'])
no_sequences = 200
sequence_length = 5

# label_map = {label:num for num, label in enumerate(actions)}

# sequences, labels = [], []
# for action in actions:
#     for sequence in np.array(os.listdir(os.path.join(DATA_PATH, action))).astype(int):
#         window = []
#         for frame_num in range(sequence_length):
#             res = np.load(os.path.join(DATA_PATH, action, str(sequence), "{}.npy".format(frame_num)))
#             window.append(res)
#         sequences.append(window)
#         labels.append(label_map[action])


left_model = Sequential()
left_model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(5,63)))
left_model.add(LSTM(128, return_sequences=True, activation='relu'))
left_model.add(LSTM(64, return_sequences=False, activation='relu'))
left_model.add(Dense(64, activation='relu'))
left_model.add(Dense(32, activation='relu'))
left_model.add(Dense(actions.shape[0], activation='softmax'))

left_model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])


right_model = Sequential()
right_model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(5,63)))
right_model.add(LSTM(128, return_sequences=True, activation='relu'))
right_model.add(LSTM(64, return_sequences=False, activation='relu'))
right_model.add(Dense(64, activation='relu'))
right_model.add(Dense(32, activation='relu'))
right_model.add(Dense(actions.shape[0], activation='softmax'))

right_model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])


left_model.load_weights('left_hand_data/test.h5')
right_model.load_weights('right_hand_data/test.h5')

from scipy import stats

sine wave

In [ ]:
def pitch_to_freq(pitch):
    return 440 * 2 ^ ((pitch-9)/12)

def freq_to_pitch(freq):
    return 12 * math.log2(freq/440) + 9

# CHORDS = {
#     0   : [-10, -6, -4],
#     0.1 : [-7, -4, -2],
#     0.2 : [-5, -2, 1],
#     0.3 : [-3, 1, 3],
#     0.4 : [-1, 3, 5],
#     0.5 : [0, 4, 7],
#     0.6 : [2, 5, 9],
#     0.7 : [3, 7, 10],
#     0.8 : [5, 9, 12],
#     0.9 : [7, 11, 14],
#     1   : [9, 13, 16]
# }

CHORDS = {
    0   : [-6, -2, 1],
    0.1 : [-5, -1, 2],
    0.2 : [-4, 0, 3],
    0.3 : [-2, 2, 5],
    0.4 : [-1, 3, 6],
    0.5 : [0, 4, 7],
    0.6 : [0.5, 5, 8],
    0.7 : [2, 6, 9],
    0.8 : [4, 8, 11],
    0.9 : [5, 9, 12],
    1   : [6, 10, 13]
}



class Waves:
    def __init__(self):
        self.wave1 = SineWave(pitch=0, pitch_per_second=100, decibels_per_second=100)
        self.wave2 = SineWave(pitch=4, pitch_per_second=100, decibels_per_second=100)
        self.wave3 = SineWave(pitch=7, pitch_per_second=100, decibels_per_second=100)
        self.waves = [self.wave1, self.wave2, self.wave3]

    def get_chords(self, x_pos):
        chord = [0, 4, 7]
        offset = 2
        difference = x_pos - 0.5

        new_chord = []
        for c in chord:
            new_chord.append(c + (difference*10) + offset)

        return new_chord

    def update_chord(self, x_pos):
        pitches = self.get_chords(x_pos)
        for i in range(3):
            self.waves[i].set_pitch(pitches[i])

        self.run()

    def set_volume(self, volume):
        for wave in self.waves:
            wave.set_volume(volume)

    def run(self):
        for wave in self.waves:
            wave.play()
        # time.sleep(1.5) # Comment out when done

    def stop(self):
        for wave in self.waves:
            wave.stop()

pygame

In [ ]:
# Initialize Pygame
pygame.mixer.init()

# Set the number of mixer channels
pygame.mixer.set_num_channels(4)
# Load the sound files
sound1 = pygame.mixer.Sound('choir.wav')
sound2 = pygame.mixer.Sound('epic-anthem-synth.wav')
sound3 = pygame.mixer.Sound('echo-synth.wav')
sound4 = pygame.mixer.Sound('back-beat.wav')

sound1.set_volume(10)
sound2.set_volume(10)
sound3.set_volume(10)
sound4.set_volume(10)



print(sound1.get_length())

channel1 = pygame.mixer.Channel(0)
channel2 = pygame.mixer.Channel(1)
channel3 = pygame.mixer.Channel(2)
channel4 = pygame.mixer.Channel(3)

# Define a function to play sounds based on input
def play_sound(input):
    if input == 1 and not channel1.get_busy():
        # channel = pygame.mixer.Channel(0)
        channel1.play(sound1)
        channel1.fadeout(int(sound1.get_length()*1000 - 2500))
    elif input == 2 and not channel2.get_busy():
        # channel = pygame.mixer.Channel(1)
        channel2.play(sound2)
        # channel2.fadeout(int(sound2.get_length()*1000 - 30))
    elif input == 3 and not channel3.get_busy():
        # channel = pygame.mixer.Channel(2)
        channel3.play(sound3)
        # channel3.fadeout(int(sound3.get_length()*1000 - 30))
    elif input == 4 and not channel4.get_busy():
        # channel = pygame.mixer.Channel(2)
        channel4.play(sound4)
        # channel3.fadeout(int(sound3.get_length()*1000 - 30))
    


5.941312313079834


In [ ]:
play_sound(4)

In [ ]:

THEREMIN_MODE = false


# 1. New detection variables
left_sequence = []
right_sequence = []

threshold = 0.5
left_sentence = 'nothing'
right_sentence = 'nothing'
cap = cv2.VideoCapture(0)

if THEREMIN_MODE:
    sine_wave = Waves()
    sine_wave.run()

y_pos = 0
x_pos = 0
sequence_length = 5

count = 0
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():

        # Read feed
        ret, frame = cap.read()

        # Make detections
        image, results = mediapipe_detection(frame, holistic)
        # print(results)
        
        # Draw landmarks
        draw_styled_landmarks(image, results)
        
        #Prediction logic
        left_keypoints, right_keypoints = extract_keypoints(results)
        left_sequence.append(left_keypoints)
        right_sequence.append(right_keypoints)

        left_sequence = left_sequence[-sequence_length:]
        right_sequence = right_sequence[-sequence_length:]
        
        # make left predictions
        if len(left_sequence) == sequence_length:
            left_res = left_model.predict(np.expand_dims(left_sequence, axis=0))[0]
            # predictions.append(np.argmax(res))
           
            # check for play sound
            left_guess = np.argmax(left_res)
            left_sentence = 'nothing'
            if left_res[left_guess] > 0.95:
                left_sentence = 'left hand'
                if left_guess == 1:
                    left_sentence = 'left fist'
                    play_sound(1)
                if left_guess == 2:
                    left_sentence = 'left thumbs up'
                    play_sound(2)

        # make right predictions
        if len(right_sequence) == sequence_length:
            right_res = right_model.predict(np.expand_dims(right_sequence, axis=0))[0]
            right_sentence = 'nothing'
            # check forplay sound
            right_guess = np.argmax(right_res)
            if right_res[right_guess] > 0.95:
                right_sentence = 'right hand'
                if right_guess == 1:
                    right_sentence = 'right fist'
                    play_sound(4)
                if right_guess == 2:
                    right_sentence = 'right thumbs up'
                    play_sound(3)
        
        # # sinth sounds
        if THEREMIN_MODE:
            if results.right_hand_landmarks != None:
                right_pos = results.right_hand_landmarks.landmark[0]
                y_pos = right_pos.y
            if results.right_hand_landmarks != None:
                right_pos = results.right_hand_landmarks.landmark[0]
                x_pos = right_pos.x

            # range 0 to 50
            y_pos = 1 - y_pos
            volume = y_pos
            sine_wave.set_volume(volume)

            # range -10 to 10
            x_pos = 1 - x_pos
            x_pos = round(x_pos, 1)
            sine_wave.update_chord(x_pos)
        
        # # Show to screen
        sentence = left_sentence + ' ' + right_sentence
        cv2.rectangle(image, (0,0), (640, 40), (245, 117, 16), -1)
        cv2.putText(image, ' '.join(sentence), (3,30), 
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)


        cv2.imshow('OpenCV Feed', image)
        count += 1
        # Break gracefully
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    sine_wave.stop()
    cv2.destroyAllWindows()

1/1 [==============================] - 0s 17ms/step


output underflow
output underflow
output underflow


1/1 [==============================] - 0s 17ms/step


In [ ]:
cap.release()